In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import random
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import BernoulliNB

In [3]:
review_raw = pd.read_csv('yelp_labelled.txt', delimiter= '\t', header=None)
review_raw.columns = ['message', 'sentement']
review_raw.head()

,message,sentement
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [4]:
keywords = [ 'only', 'back', 'very', 'over', 'food', 'took', 'horrible', 'don\'t', 'least', 'much', 'excuse', 'down', 'bad', 'any', 'sucks', 'final', 'worst', 'hate', 'hated', 'never','not', 'slow', 'nasty', 'angry', 'didn\'t', 'poor', 'no']

In [5]:
for key in keywords:
    # Note that we add spaces around the key so that we're getting the word,
    # not just pattern matching.
    review_raw[str(key)] = review_raw.message.str.contains(
        ' ' + str(key) + ' ',
        case=False
    )
    
review_raw['allcaps'] = review_raw.message.str.isupper()

data = review_raw[keywords + ['allcaps']]
target = review_raw['sentement']

# Instantiate our model and store it in a new variable.
bnb = BernoulliNB()

In [16]:
#divid the data and the target into ten diffent, without overlaping, datasets.
datafold = data
targetfold = target
d={}
t={}
for i in range(0, 10):
    d['datafoldsample{}'.format(i)] = datafold.sample(n=100, replace=False)
    indexNum = d['datafoldsample{}'.format(i)].index.tolist()
    datafold = datafold.drop(indexNum)
    
    t['targetfoldsample{}'.format(i)] = targetfold.take(indexNum)

In [9]:
#Go through each of the ten equaly divided data and target sets and create traing and testing sets
print(cross_val_score(bnb, data, target, cv=10))
for sample in d:
    X_train = pd.DataFrame()
    Y_train = pd.DataFrame()
    for x in range(0, 10):
        if sample != 'datafoldsample{}'.format(x):
            X_train = X_train.append(d['datafoldsample{}'.format(x)])
            Y_train = pd.concat([Y_train, t['targetfoldsample{}'.format(x)]])
        else:
            X_test = d['datafoldsample{}'.format(x)]
            Y_test = t['targetfoldsample{}'.format(x)]
    
    y_pred = bnb.fit(X_train, Y_train).predict(X_test)
    
    k = confusion_matrix(Y_test, y_pred)
    print('With 20% Holdout: ' + str(100*bnb.fit(X_train, Y_train).score(X_test, Y_test)))
    print(k)
    print('percentage of true positive: ' + str(100*k[0][0]/(k[0][0] + k[0][1])))
    print('percentage of true negative: ' + str(100*k[1][1]/(k[1][0] + k[1][1])) + '\n')

[ 0.72  0.62  0.72  0.59  0.67  0.64  0.69  0.66  0.62  0.66]
With 20% Holdout: 62.0
[[14 33]
 [ 5 48]]
percentage of true positive: 29.7872340426
percentage of true negative: 90.5660377358

With 20% Holdout: 61.0
[[19 35]
 [ 4 42]]
percentage of true positive: 35.1851851852
percentage of true negative: 91.3043478261

With 20% Holdout: 70.0
[[19 26]
 [ 4 51]]
percentage of true positive: 42.2222222222
percentage of true negative: 92.7272727273

With 20% Holdout: 70.0
[[24 28]
 [ 2 46]]
percentage of true positive: 46.1538461538
percentage of true negative: 95.8333333333

With 20% Holdout: 69.0
[[18 25]
 [ 6 51]]
percentage of true positive: 41.8604651163
percentage of true negative: 89.4736842105

With 20% Holdout: 64.0
[[14 31]
 [ 5 50]]
percentage of true positive: 31.1111111111
percentage of true negative: 90.9090909091

With 20% Holdout: 69.0
[[24 25]
 [ 6 45]]
percentage of true positive: 48.9795918367
percentage of true negative: 88.2352941176



C:\Users\ktser\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


With 20% Holdout: 61.0
[[19 36]
 [ 3 42]]
percentage of true positive: 34.5454545455
percentage of true negative: 93.3333333333

With 20% Holdout: 65.0
[[26 31]
 [ 4 39]]
percentage of true positive: 45.6140350877
percentage of true negative: 90.6976744186

With 20% Holdout: 66.0
[[24 29]
 [ 5 42]]
percentage of true positive: 45.2830188679
percentage of true negative: 89.3617021277



### Above is evaluation of my first classifier model. The performance based on accuracy. There overfitting due to the difference in perforance of training and test data sets and the inconsistancy between the folds. Lot of the error is due to false positive, our feature set is more email as spam than ham. Although we are perdicting hams at very high percentage, we are letting in alot of spams, so I should focus more on feature engeneering to correctly perdict spams.

In [10]:
keywords2 = [ 'only', 'very', 'over', 'don\'t', 'least', 'took', 'much', 'down', 'bad', 'any', 'hate', 'hated', 'never','not', 'didn\'t', 'no']

In [20]:
review_raw = pd.read_csv('yelp_labelled.txt', delimiter= '\t', header=None)
review_raw.columns = ['message', 'sentement']

for key in keywords2:
    # Note that we add spaces around the key so that we're getting the word,
    # not just pattern matching.
    review_raw[str(key)] = review_raw.message.str.contains(
        ' ' + str(key) + ' ',
        case=False
    )

data2 = review_raw[keywords2]

datafold = data2
targetfold = target
d={}
t={}
for i in range(0, 10):
    d['datafoldsample{}'.format(i)] = datafold.sample(n=100, replace=False)
    indexNum = d['datafoldsample{}'.format(i)].index.tolist()
    datafold = datafold.drop(indexNum)
    
    t['targetfoldsample{}'.format(i)] = targetfold.take(indexNum)
    

print(cross_val_score(bnb, data2, target, cv=10))

for sample in d:
    X_train = pd.DataFrame()
    Y_train = pd.DataFrame()
    for x in range(0, 10):
        if sample != 'datafoldsample{}'.format(x):
            X_train = X_train.append(d['datafoldsample{}'.format(x)])
            Y_train = pd.concat([Y_train, t['targetfoldsample{}'.format(x)]])
        else:
            X_test = d['datafoldsample{}'.format(x)]
            Y_test = t['targetfoldsample{}'.format(x)]
    
    y_pred = bnb.fit(X_train, Y_train).predict(X_test)
    
    k = confusion_matrix(Y_test, y_pred)
    print('With 20% Holdout: ' + str(100*bnb.fit(X_train, Y_train).score(X_test, Y_test)))
    print(k)
    print('percentage of true positive: ' + str(100*k[0][0]/(k[0][0] + k[0][1])))
    print('percentage of true negative: ' + str(100*k[1][1]/(k[1][0] + k[1][1])) + '\n')

[ 0.72  0.64  0.71  0.59  0.65  0.61  0.66  0.62  0.6   0.66]
With 20% Holdout: 68.0
[[15 29]
 [ 3 53]]
percentage of true positive: 34.0909090909
percentage of true negative: 94.6428571429

With 20% Holdout: 61.0
[[21 34]
 [ 5 40]]
percentage of true positive: 38.1818181818
percentage of true negative: 88.8888888889

With 20% Holdout: 69.0
[[20 30]
 [ 1 49]]
percentage of true positive: 40.0
percentage of true negative: 98.0

With 20% Holdout: 58.0
[[14 36]
 [ 6 44]]
percentage of true positive: 28.0
percentage of true negative: 88.0

With 20% Holdout: 67.0
[[19 29]
 [ 4 48]]
percentage of true positive: 39.5833333333
percentage of true negative: 92.3076923077

With 20% Holdout: 66.0
[[22 30]
 [ 4 44]]
percentage of true positive: 42.3076923077
percentage of true negative: 91.6666666667

With 20% Holdout: 65.0

C:\Users\ktser\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



[[22 33]
 [ 2 43]]
percentage of true positive: 40.0
percentage of true negative: 95.5555555556

With 20% Holdout: 63.0
[[19 36]
 [ 1 44]]
percentage of true positive: 34.5454545455
percentage of true negative: 97.7777777778

With 20% Holdout: 65.0
[[17 31]
 [ 4 48]]
percentage of true positive: 35.4166666667
percentage of true negative: 92.3076923077

With 20% Holdout: 65.0
[[14 29]
 [ 6 51]]
percentage of true positive: 32.5581395349
percentage of true negative: 89.4736842105



### For the second feature set I removed all the words that are not adverb. The result did not improve, overfitting is lager and the type of error contributing to over all error is random between the folds. The type of error being random might be due to imbalance is the samples for the folds.

In [14]:
k = review_raw[review_raw.sentement == 0].reset_index().drop('index', axis=1)
negFeatureslist = []

for i in k['message']:
    c = i.split(' ')
    for x in c:
        negFeatureslist.append(x)

#negFeatureslist = list(negFeatureslist)
negFeatureslist = ','.join([i for i in negFeatureslist if i.isalpha()])
negFeatureslist = negFeatureslist.split(',')

### The third feature set includes all the words in the spam email, excluding the words also found in the ham email.

In [19]:
for key in negFeatureslist:
    # Note that we add spaces around the key so that we're getting the word,
    # not just pattern matching.
    review_raw[str(key)] = review_raw.message.str.contains(
        ' ' + str(key) + ' ',
        case=False
    )

data3 = review_raw[negFeatureslist]

datafold = data3
targetfold = target
d={}
t={}
for i in range(0, 10):
    d['datafoldsample{}'.format(i)] = datafold.sample(n=100, replace=False)
    indexNum = d['datafoldsample{}'.format(i)].index.tolist()
    datafold = datafold.drop(indexNum)
    
    t['targetfoldsample{}'.format(i)] = targetfold.take(indexNum)

print(cross_val_score(bnb, data3, target, cv=10))

for sample in d:
    X_train = pd.DataFrame()
    Y_train = pd.DataFrame()
    for x in range(0, 10):
        if sample != 'datafoldsample{}'.format(x):
            X_train = X_train.append(d['datafoldsample{}'.format(x)])
            Y_train = pd.concat([Y_train, t['targetfoldsample{}'.format(x)]])
        else:
            X_test = d['datafoldsample{}'.format(x)]
            Y_test = t['targetfoldsample{}'.format(x)]
    
    y_pred = bnb.fit(X_train, Y_train).predict(X_test)
    
    k = confusion_matrix(Y_test, y_pred)
    print('With 20% Holdout: ' + str(100*bnb.fit(X_train, Y_train).score(X_test, Y_test)))
    print(k)
    print('percentage of true positive: ' + str(100*k[0][0]/(k[0][0] + k[0][1])))
    print('percentage of true negative: ' + str(100*k[1][1]/(k[1][0] + k[1][1])) + '\n')

[ 0.65  0.67  0.7   0.62  0.66  0.67  0.58  0.69  0.55  0.62]


C:\Users\ktser\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


With 20% Holdout: 57.0
[[22 28]
 [15 35]]
percentage of true positive: 44.0
percentage of true negative: 70.0

With 20% Holdout: 68.0
[[24 23]
 [ 9 44]]
percentage of true positive: 51.0638297872
percentage of true negative: 83.0188679245

With 20% Holdout: 65.0
[[23 26]
 [ 9 42]]
percentage of true positive: 46.9387755102
percentage of true negative: 82.3529411765

With 20% Holdout: 61.0
[[23 25]
 [14 38]]
percentage of true positive: 47.9166666667
percentage of true negative: 73.0769230769

With 20% Holdout: 61.0
[[23 27]
 [12 38]]
percentage of true positive: 46.0
percentage of true negative: 76.0

With 20% Holdout: 62.0
[[30 23]
 [15 32]]
percentage of true positive: 56.6037735849
percentage of true negative: 68.085106383

With 20% Holdout: 75.0
[[27 16]
 [ 9 48]]
percentage of true positive: 62.7906976744
percentage of true negative: 84.2105263158

With 20% Holdout: 63.0
[[29 29]
 [ 8 34]]
percentage of true positive: 50.0
percentage of true negative: 80.9523809524

With 20% Holdo